In [9]:
import os, json
from dotenv import load_dotenv

load_dotenv()

with open(f"/{os.getenv("WDIR")}/scrape/storage/woolworths_cats.json") as f:
    woolworths_cat_data = json.load(f)[1:21]

with open(f"/{os.getenv("WDIR")}/scrape/storage/coles_cats.json") as f:
    coles_cat_data = json.load(f)


In [33]:
import json

json.loads(woolworths_cat_data[0])
# json.loads(coles_cat_data[1])['catalogGroupView']

{'NodeId': '1_DEF0CCD',
 'Description': 'Christmas',
 'UrlFriendlyName': 'christmas',
 'Children': [{'NodeId': '1_DEF0CCD_SPECIALS',
   'Description': 'Christmas Specials',
   'UrlFriendlyName': 'christmas-specials',
   'Children': []},
  {'NodeId': '1_1C11EF0',
   'Description': 'Woolworths Festive Range',
   'UrlFriendlyName': 'woolworths-festive-range',
   'Children': [{'NodeId': '1_1C11EF0_SPECIALS',
     'Description': 'Woolworths Festive Range Specials',
     'UrlFriendlyName': 'woolworths-festive-range-specials',
     'Children': []},
    {'NodeId': '1_DEC0B71',
     'Description': 'Perfect Platter Essentials',
     'UrlFriendlyName': 'perfect-platter-essentials',
     'Children': []},
    {'NodeId': '1_794CC11',
     'Description': 'Sides & Mains',
     'UrlFriendlyName': 'sides-mains',
     'Children': []},
    {'NodeId': '1_6F4CD0F',
     'Description': 'Festive Sweets & Treats',
     'UrlFriendlyName': 'festive-sweets-treats',
     'Children': []},
    {'NodeId': '1_C27E482'

## psycopg2

### Database Connect

Connect to an existing database, creating a new database session for which methods for th

In [ ]:
import psycopg2

# conn = psycopg2.connect(**params)

### Cursor

Creates a cursor instance encapsulated by the connection to perform database operations.

create new cursor instances using the cursor() method to execute database commands and queries.

In [24]:
# Open a cursor to perform database operations

cur = conn.cursor()

### Executing Commands

The class cursor allows interaction with the database:

send commands to the database using methods such as execute() and executemany(),

retrieve data from the database by iteration or using methods such as fetchone(), fetchmany(), fetchall().

In [30]:
def process_coles_cat(cat, parentId=None):
    cur.execute("""
        INSERT INTO category (catid, parentid, catname, companyid, other)
        VALUES (%s, %s, %s, %s, %s);
    """,
    (cat['id'], parentId, cat['name'], 2, cat['originalName'])
    )

    for child in cat['catalogGroupView']:
        process_coles_cat(child, parentId=cat['id'])


for cat in coles_cat_data:
    process_coles_cat(json.loads(cat))    

conn.commit()

In [41]:
def process_woolworths_cat(cat, parentId=None):
    cur.execute("""
        INSERT INTO category (catid, parentid, catname, companyid, other)
        VALUES (%s, %s, %s, %s, %s);
    """,
    (cat['NodeId'], parentId, cat['Description'], 1, cat['UrlFriendlyName'])
    )

    for child in cat['Children']:
        process_woolworths_cat(child, parentId=cat['NodeId'])


for cat in woolworths_cat_data:
    process_woolworths_cat(json.loads(cat))    

conn.commit()

In [48]:
cur.execute("""
    SELECT catid, other
    FROM category 
    WHERE companyid = 2
""")

In [49]:
vals = cur.fetchall()

In [72]:
chars = set()

for _, other in vals:
    for c in other:
        if (ord('a') <= ord(c) <= ord('z')) or (ord('A') <= ord(c) <= ord('Z')):
            pass
        else:
            chars.add(c)


In [76]:
chars.remove("2")

{' ', '&', ',', '-', '/'}

In [77]:
for id, other in vals:
    for index, c in enumerate(other):
        if c in chars:
            other[index] = '-'


TypeError: 'str' object does not support item assignment